## Model inference

In this notebook, we will load a previously trained model, explore the learned topics, and predict topics for a paper on arXiv.

In [1]:
# imports
import sys
sys.path.insert(0, "../")
from utils import scrape_arxiv_abstract
from model import TopicModel
from dataset import ArXivDataset
from gensim.models import LdaModel
from pprint import pprint
import PyPDF2
import plotly.express as px
import pandas as pd

### Build topic model

To build a `TopicModel` object, we need to pass in as arguments the dataset used to create the model (to process new instances) and the model itself (to predict topics for the new instances).

In [2]:
# create topic model
model_path = "../models/lda_n38_p5_r929_c37.1"
dataset_path = "../object/dataset.obj"
model = TopicModel(model_path, dataset_path)

### Investigate topics

Next, let us explore the different topics learned by the model so that we can assign understandable topic names to each cluster.

In [3]:
# print topics
pprint(model.topics)

[(0,
  [('control', 0.024879357),
   ('model', 0.016764874),
   ('system', 0.016609652),
   ('course', 0.014260766),
   ('work', 0.011419694),
   ('skill', 0.011288833),
   ('law', 0.010282257),
   ('lab', 0.0098598),
   ('student', 0.008887271),
   ('problem', 0.008220088)]),
 (1,
  [('system', 0.039077986),
   ('course', 0.02025457),
   ('model', 0.0118238665),
   ('student', 0.011125353),
   ('project', 0.010907774),
   ('design', 0.010358624),
   ('campus', 0.010149597),
   ('engineering', 0.007867761),
   ('problem', 0.0074642575),
   ('component', 0.007119093)]),
 (2,
  [('course', 0.031964216),
   ('information', 0.0133742),
   ('communication', 0.011183536),
   ('digital', 0.01115881),
   ('software', 0.00955001),
   ('student', 0.0093846135),
   ('system', 0.009206128),
   ('channel', 0.008912577),
   ('application', 0.008868147),
   ('exam', 0.0086734975)]),
 (3,
  [('system', 0.032837804),
   ('course', 0.020685153),
   ('control', 0.014234064),
   ('application', 0.01073114

In [4]:
topicName = ''
TopicsNames =[]
for i in model.topics:
    for name,prob in i[1][:5]:
        topicName = topicName + name + ' ' 
    TopicsNames.append(topicName)
    topicName = ''

In [6]:
model.set_topic_names(TopicsNames)
pprint(model.topics)

[(' control model system course work',
  '0.025*"control" + 0.017*"model" + 0.017*"system" + 0.014*"course" + '
  '0.011*"work" + 0.011*"skill" + 0.010*"law" + 0.010*"lab" + 0.009*"student" '
  '+ 0.008*"problem"'),
 (' system course model student project',
  '0.039*"system" + 0.020*"course" + 0.012*"model" + 0.011*"student" + '
  '0.011*"project" + 0.010*"design" + 0.010*"campus" + 0.008*"engineering" + '
  '0.007*"problem" + 0.007*"component"'),
 (' course information communication digital software',
  '0.032*"course" + 0.013*"information" + 0.011*"communication" + '
  '0.011*"digital" + 0.010*"software" + 0.009*"student" + 0.009*"system" + '
  '0.009*"channel" + 0.009*"application" + 0.009*"exam"'),
 (' system course control application student',
  '0.033*"system" + 0.021*"course" + 0.014*"control" + 0.011*"application" + '
  '0.010*"student" + 0.009*"design" + 0.008*"campus" + 0.007*"modeling" + '
  '0.007*"development" + 0.007*"android"'),
 (' project course thay resource student'

We can see that there are some clusters that seem to refer to specific topics in machine learning. One of them is topic 7, which seems to direcly relate to sequential and time-series data. Another example is topic 10, which seems to be related to reinforcement learning.

To make it easier to refer to these topic clusters, we will assign (tentative) names to each of them. Note that these names are subject to interpretation and are only assigned to help "summarize" each cluster.

### Predict topics for a paper

Let us see how our model predicts a paper taken directly from arXiv. Using the `scrape_arxiv_abstract()` function, we can extract the title and the abstract of any paper on arXiv given its URL. Once scraped, this title and abstract can be passed into our topic model's `predict()` method.

To illustrate, let us scrape the title and abstract from the seminal paper ["Attention Is All You Need" (2017)](https://arxiv.org/abs/1706.03762) and see what topics the model detects.

In [7]:
text = "The physical systems generally rely on the fundamental concept of a feedback loop, allowing them to be controlled and giving them a behavior that is as insensitive as possible to environmental disturbances. The general objective of this course is to provide students with the concepts and skills enabling them to understand the structure and interactions within existing dynamic systems or along with their design. They will also be able to process information, design a control law to meet specifications, and analyze its performance and robustness. To achieve this, the students must first be able to define a model (or a set of models), highlighting the variables influencing the state of this system (inputs), the measures allowing access to this state, and variables to which the specifications relate (outputs), as well as the relationships between these variables. Then, in a second step, and from the analysis of the inputs that can be controlled (commands) or those that are undergone (disturbances), students will have to design a control law in order to ensure the expected performances. The last step in this course will concern the analysis of the robustness of the determined control law."

In [8]:
# get predictions
predictions = model.predict(text)

In [9]:
Top5Topics, Top5Probabilities = model.predictToChart(text)

In [10]:
fig = px.line_polar(r=Top5Probabilities, theta=Top5Topics, line_close=True)
fig.show()

/home/vitoriano/.local/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])


In [11]:
text2 = "The goal of this training is to let you discover the different stages of an aircraft design process in both a theoretical and a practical perspective. You will be introduced to the typical methods used in an aircraft design office, and apply this knowledge by doing the preliminary design of your own aircraft. After completing this training course, you will have acquired knowledge and skills that will enable you to work out the main aircraft characteristics and layout in a very short time frame."


In [12]:
Top5Topics, Top5Probabilities = model.predictToChart(text2)

In [13]:
fig = px.line_polar(r=Top5Probabilities, theta=Top5Topics, line_close=True)
fig.show()

/home/vitoriano/.local/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

